<a href="https://colab.research.google.com/github/SpirinEgor/gulag/blob/master/notebooks/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture

from os import environ

# Set this to `True` to train on Colab
ON_CLOUD = True

if ON_CLOUD:
    from getpass import getpass
    from urllib.parse import quote

    token = getpass("GitHub token: ")
    token = quote(token)
    environ["GITHUB_TOKEN"] = token
    ! if [ -d gulag ]; then rm -Rf gulag; fi
    ! git clone https://$GITHUB_TOKEN@github.com/SpirinEgor/gulag.git
    %cd gulag
    ! pip install -q -r requirements.txt
else:
    %cd ..

GitHub token: ··········


In [2]:
import gin

from src.data import MultiLanguageClassificationDataModule
from src.main import train
from src.utils import setup_logging

setup_logging()

In [11]:
gin.parse_config_file("config/train.gin")

print(gin.config_str())

import gin.torch.external_configurables
import src.utils

# Parameters for AdamW:
# ==============================================================================
AdamW.lr = 0.001
AdamW.weight_decay = 0.0

# Parameters for configure_optimizers:
# ==============================================================================
configure_optimizers.optimizer_cls = @AdamW
configure_optimizers.scheduler_cls = @LambdaLR

# Parameters for generate_eval_samples:
# ==============================================================================
generate_eval_samples.n_samples = 10000

# Parameters for generate_example:
# ==============================================================================
generate_example.max_langs = 10
generate_example.max_samples_per_lang = 10
generate_example.max_seq_len = 512
generate_example.min_langs = 1

# Parameters for LambdaLR:
# ==============================================================================
LambdaLR.lr_lambda = @rsqrt_with_warmup

# Parameters 

# Data overview

Some examples from synthetic dataset

In [12]:
data_module = MultiLanguageClassificationDataModule(batch_size=1)
data_module.setup()

INFO:src.data.data_module:Downloading and opening 'wikiann' dataset for ru, uk, be, kk, az, hy, ka, he, en, de
INFO:datasets.info:Loading Dataset Infos from /root/.cache/huggingface/modules/datasets_modules/datasets/wikiann/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e
INFO:datasets.builder:Overwrite dataset info from restored data version.
INFO:datasets.info:Loading Dataset info from /root/.cache/huggingface/datasets/wikiann/ru/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e
INFO:datasets.info:Loading Dataset info from /root/.cache/huggingface/datasets/wikiann/ru/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e
INFO:datasets.info:Loading Dataset Infos from /root/.cache/huggingface/modules/datasets_modules/datasets/wikiann/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e
INFO:datasets.builder:Overwrite dataset info from restored data version.
INFO:datasets.info:Loading Dataset info from /root/.cache/hugging

In [13]:
for i, batch in enumerate(data_module.train_dataloader()):
    if i == 5:
        break
    seq, attn, labels = batch

    orig_str = data_module.tokenizer.decode(seq[0])
    class_names = data_module.decode_languages(labels[0])

    print(
        f"""{'=' * 20}
Input tokens (len = {seq.shape[-1]}):
{seq[0]}
Target classes:
{class_names}
Original string:
{orig_str}
{'=' * 20}
"""
    )

Input tokens (len = 441):
tensor([   101,    509, 111069,  22681,  89338,    143,  11490,  15417,    543,
         42714,  66055,  12903,  64429,    731,  14340,  25941,  20189,  58811,
         22622,  55920,  89887,  10191,    599,  16481,  23444,  36656,  42546,
         15773,    549,  12265,  49223,    522, 111102,  10778,  30820,  53607,
           142,  10990,  25109,  53272,    539,  26720,  59509,  56269,  26804,
           521,  13097,  11347,  38368,  11184,  14259,  20700,  10387,  78099,
         15634,    160,  11259,  37611,  24093,  51036,  37611,  69849,  11447,
         10925, 100078,  21189, 108929,    519,  31016,  80350,  11427,  53824,
           541,  94454,  49569,  11890,  21220,  62903,  14797,  44503,  10205,
         25782,    509,  50301, 106714,  20634,  39076,   1575,  11577,  39362,
          1589,  13266,  54519,  30508,  20628,  13266,  32414,   1572,  42382,
        108894,  17988,    510,  15768,  36656,    530,  45209,  10316,  10211,
         42687

# Train model

In [14]:
train()

Global seed set to 7


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmpzlg3mo75
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmpzlg3mo75/_remote_module_non_sriptable.py
loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "

Sanity Checking: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1445: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  f"`.{fn}(ckpt_path=None)` was called without a model."
INFO:src.data.data_module:Downloading and opening 'wikiann' dataset for ru, uk, be, kk, az, hy, ka, he, en, de
INFO:datasets.info:Loading Dataset Infos from /root/.cache/huggingface/modules/datasets_modules/datasets/wikiann/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e
INFO:datasets.builder:Overwrite dataset info from restored data version.
INFO:datasets.info:Loading Dataset info from /root/.cache/huggingface/datasets/wikiann/ru/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e
INFO:datasets.info:Loading Dataset info f

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         test/f1            0.9415807723999023
        test/loss           0.18368224799633026
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr-AdamW,███▆▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test/f1,▁
test/loss,▁
train/f1,▁
train/loss,▁
train/step_f1,▁▅▄▆▅▅▆▅▅▇▇▆▆▆▇▇▅▆▅█▅▇▅▇█▇▅▆██▇▇▆▆▇█▇▇▇▆
train/step_loss,█▄▅▃▄▄▃▃▄▂▂▂▂▃▂▂▃▂▃▂▃▁▃▂▁▂▃▂▁▁▂▂▂▃▂▁▁▂▂▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val/f1,▁▃▄▅▆▆▆▇▇▇▇▇█▇▇█████
val/loss,█▅▄▃▃▂▂▁▂▂▂▁▁▃▃▂▁▂▁▂
